# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
J. Liu  ->  J. Liu  |  ['J. Liu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
S. Vaughan  ->  S. Vaughan  |  ['S. Vaughan']
K. Lee  ->  K. Lee  |  ['K. Lee']
N. Martin  ->  N. Martin  |  ['N. Martin']


Arxiv has 81 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.16018


extracting tarball to tmp_2502.16018...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']


...

/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure 
          linearPS_gr0color_idx_eff_temp_relBHBstarsBarbosa_et_al2022
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics{
          linearPS_gr0color_idx_eff_temp_relBHBstarsBarbosa_et_al2022}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure comparisonLOS_vel_estimates_vel_sts_appendix
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure comparisonLOS_vel_estimates_vel_sts_appendix as tm

/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure target_selection
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure target_selection as tmp_2502.17319/./target_selection.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure vel_struct_dist_trail_armSgr_stream_main
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure vel_struct_dist_trail_armSgr_stream_main as tmp_2502.17319/./vel_struct_dist_trail_armSgr_stream_main.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure resolving_power
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure resolving_power as tmp_2502.17319/./resolving_power.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 169 bibliographic references in tmp_2502.17319/eso_programs_p106_21l8run1and2_paper_v_arxiv.bbl.
Error retrieving bib data for ps1: 'author'
Error retrieving bib data for  magnier_et_al2020: 'author'
Error retrieving bib data for sdss_dr18: 'author'
Error retrieving bib data for starkenburg_et_al2017: 'author'
Error retrieving bib data for vasiliev_et_al2021: 'author'
Error retrieving bib data for dierickx_loeb2017: 'author'
Error retrieving bib data for vasiliev_et_al2021: 'author'
Error retrieving bib data for vasiliev_baumgardt2021: 'author'
Error retrieving bib data for vasiliev_baumgardt2021: 'author'
Error retrieving bib data for baumgardt_vasiliev2021: 'author'
Error retrieving bib data for xue_et_al2008: 'author'
Error retrieving bib data for xue_et_al2011: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.16018-b31b1b.svg)](https://arxiv.org/abs/2502.16018) | **Application of resolved low-J multi-CO line modeling with RADEX to constrain the molecular gas properties in the starburst M82**  |
|| V. Zhang, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-02-25*|
|*Comments*| *14 pages, 6 figures, accepted for publication in ApJ*|
|**Abstract**|            The distribution and physical conditions of molecular gas are closely linked to star formation and the subsequent evolution of galaxies. Emission from carbon monoxide (CO) and its isotopologues traces the bulk of molecular gas and provides constraints on the physical conditions through their line ratios. However, comprehensive understanding on how the particular choice of line modeling approach impacts derived molecular properties remain incomplete. Here, we study the nearby starburst galaxy M82, known for its intense star formation and molecular emission, using the large set of available multi-CO line observations. We present high-resolution (${\sim}85$ pc) emission of seven CO isotopologue lines, including $^{12}$CO, $^{13}$CO, and C$^{18}$O from the $J = 1-0$, $2-1$ and $3-2$ transitions. Using \texttt{RADEX} for radiative transfer modeling, we analyze M82\textsc{\char39}s molecular properties with (i) a one-zone model and (ii) a variable density model, comparing observed and simulated emissions via a minimum $\chi^2$ analysis. We find that inferred gas conditions -- kinetic temperature and density -- are consistent across models, with minimal statistical differences. However, due to their low critical densities (${<}10^{4}$ cm$^{-3}$), low-$J$ CO isotopologue lines do not effectively probe higher density gas prevalent in starburst environments like that of M82. Our results further imply that this limitation extends to high-redshift ($z{\gtrapprox}1$) galaxies with similar conditions, where low-$J$ CO lines are inadequate for density constraints. Future studies of extreme star-forming regions like M82 will require higher-$J$ CO lines or alternative molecular tracers with higher critical densities.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.17319-b31b1b.svg)](https://arxiv.org/abs/2502.17319) | **A Pristine-UNIONS view on the Galaxy: Kinematics of the distant spur feature of the Sagittarius stream traced by Blue Horizontal Branch stars**  |
|| M. Bayer, et al. -- incl., <mark>N. Martin</mark> |
|*Appeared on*| *2025-02-25*|
|*Comments*| *21 pages, 12 figures, submitted to A&A, abridged abstract*|
|**Abstract**|            Providing a detailed picture of the Sagittarius (Sgr) stream offers important constraints on the build-up of the Galactic halo as well as its gravitational potential at large radii. While several attempts have been made to model the structure of the Sgr stream, no model has yet been able to match all the features observed for the stream. Moreover, for several of these features, observational characterisation of their properties is rather limited, particularly at large distances. The aim of this work is to investigate the kinematics of the Sgr stream outermost spur feature using blue horizontal branch (BHB) stars. Candidate BHB stars were selected by combining two approaches; one capitalising on Pan-STARRS1 3$\Pi$ griz and u photometry taken as part of UNIONS, the other using Pristine Survey CaHK and SDSS ugr photometry. Follow-up optical spectra are obtained using ESO/VLT/FORS2 to confirm their BHB nature and obtain line-of-sight (LOS) velocities. Of our 25 candidates, 20 stars can be confirmed as bona fide BHB stars. Their LOS velocities, together with the 3D positions of these stars qualitatively match well with Sgr model predictions and trace the outer apocentre of the trailing arm and its spur feature very nicely. The quantitative offsets that are found between our data and the different models can be used to provide information about the Galactic gravitational potential at large distances. We present a first, tentative, analysis in this direction, showing that the model of Vasiliev et al. (2021) would provide better agreement with our observations if the enclosed mass of the Milky Way within 100 kpc were lowered to $(5.3\!\pm\!0.4)\!\times\!10^{11}$ M$_\odot$ (versus $(5.6\!\pm\!0.4)\!\times\!10^{11}$ M$_\odot$). Our selection of BHB stars provides a new view on the outermost structure in 3D positions and LOS velocities of the Sgr debris.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.16087-b31b1b.svg)](https://arxiv.org/abs/2502.16087) | **TOI-6324b: An Earth-Mass Ultra-Short-Period Planet Transiting a Nearby M Dwarf**  |
|| R. A. Lee, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2025-02-25*|
|*Comments*| *16 pages, 5 figures, 2 tables*|
|**Abstract**|            We report the confirmation of TOI-6324 b, an Earth-sized (1.059 $\pm$ 0.041 R$_\oplus$) ultra-short-period (USP) planet orbiting a nearby ($\sim$20 pc) M dwarf. Using the newly commissioned Keck Planet Finder (KPF) spectrograph, we have measured the mass of TOI-6324 b 1.17 $\pm$ 0.22 M$_\oplus$. Because of its extremely short orbit of just $\sim$6.7 hours, TOI-6324 b is intensely irradiated by its M dwarf host, and is expected to be stripped of any thick, H/He envelope. We were able to constrain its interior composition and found an iron core mass fraction (CMF = 27$\pm$37%) consistent with that of Earth ($\sim$33%) and other confirmed USPs. TOI-6324 b is the closest to Earth-sized USP confirmed to date. TOI-6324 b is a promising target for JWST phase curve and secondary eclipse observations (Emission Spectroscopy Metric = 25) which may reveal its surface mineralogy, day-night temperature contrast, and possible tidal deformation. From 7 sectors of TESS data, we report a tentative detection of the optical phase curve variation with an amplitude of 42$\pm$28 ppm.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.16413-b31b1b.svg)](https://arxiv.org/abs/2502.16413) | **Dense Cores in IRDC G14.225-0.506 revealed by ALMA Observations**  |
|| Y. Zhao, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-02-25*|
|*Comments*| **|
|**Abstract**|            Dense cores in massive, parsec-scale molecular clumps are sites that harbor protocluster formation. We present results from observations towards a hub-filament structure of a massive Infrared Dark Cloud (IRDC) G14.225-0.506 using the Atacama Large Millimeter/submillimeter Array (ALMA). The dense cores are revealed by the 1.3 mm dust continuum emission at an angular resolution of $\sim$ 1.5'' and are identified through the hierarchical Dendrogram technique. Combining with the N$_2$D$^+$ 3-2 spectral line emission and gas temperatures derived from a previous NH$_3$ study, we analyze the thermodynamic properties of the dense cores. The results show transonic and supersonic-dominated turbulent motions. There is an inverse correlation between the virial parameter and the column density, which implies that denser regions may undergo stronger gravitational collapse. Molecular outflows are identified in the CO 2-1 and SiO 5-4 emission, indicating active protostellar activities in some cores. Besides these star formation signatures revealed by molecular outflows in the dense cores, previous studies in the infrared, X-ray, and radio wavelengths also found a rich and wide-spread population of young stellar objects (YSOs), showing active star formation both inside and outside of the dense cloud.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.16455-b31b1b.svg)](https://arxiv.org/abs/2502.16455) | **Asteroid shape inversion with light curves using deep learning**  |
|| Y. Tang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-02-25*|
|*Comments*| **|
|**Abstract**|            Asteroid shape inversion using photometric data has been a key area of study in planetary science and astronomical this http URL, the current methods for asteroid shape inversion require extensive iterative calculations, making the process time-consuming and prone to becoming stuck in local optima. We directly established a mapping between photometric data and shape distribution through deep neural networks. In addition, we used 3D point clouds to represent asteroid shapes and utilized the deviation between the light curves of non-convex asteroids and their convex hulls to predict the concave areas of non-convex asteroids. We compared the results of different shape models using the Chamfer distance between traditional methods and ours and found that our method performs better, especially when handling special shapes. For the detection of concave areas on the convex hull, the intersection over union (IoU) of our predictions reached 0.89. We further validated this method using observational data from the Lowell Observatory to predict the convex shapes of the asteroids 3337 Milo and 1289 Kuta, and conducted light curve fitting experiments. The experimental results demonstrated the robustness and adaptability of the method         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.16458-b31b1b.svg)](https://arxiv.org/abs/2502.16458) | **Transformer-based Approach for Accurate Asteroid Spectra taxonomy and albedo estimation**  |
|| Y. Tang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-02-25*|
|*Comments*| **|
|**Abstract**|            China plans to launch a probe (Tianwen-2) around 2025, mainly for exploring the near-Earth asteroid 2016 HO3 . The mission involves close-range exploration, landing, and mining operations that require three-dimensional modeling of the asteroid, which requires prior knowledge of its material composition and uniformity. This information is crucial in progressive or ground exploration processes. Our research focuses on high-precision intelligent inversion of complex physical properties of asteroids based on spectral data, providing support for further analysis of aster oid materials, density, and structure. We have developed a platform for asteroid spectral classification, albedo estimation, and composition analysis, which includes three types of neural networks based on Transformer attention mechanism: One for spectral classification, achieving a four-class classification accuracy of 97.28% and an eleven-class classification accuracy of 95.69%; second one for albedo estimation, with an average absolute error of 0.0308 in S-type asteroid albedo estimation, and the third one for composition analysis, with a predicted spectral angular distance of only 0.0340 and a root mean square error of 0.1759 for the abundance of end members. These results indicate that our network can provide high-precision asteroid spectral classification, albedo estimation, and composition analysis results. In addition, we utilized the platform to analyze and provide results for six asteroids.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.16751-b31b1b.svg)](https://arxiv.org/abs/2502.16751) | **The MAGPI Survey: the kinematic morphology-density relation (or lack thereof) and the Hubble sequence at $z\sim0.3$**  |
|| C. Foster, et al. -- incl., <mark>S. Vaughan</mark> |
|*Appeared on*| *2025-02-25*|
|*Comments*| *23 pages, 19 figures, 3 tables, accepted for publication in PASA*|
|**Abstract**|            This work presents visual morphological and dynamical classifications for 637 spatially resolved galaxies, most of which are at intermediate redshift ($z\sim0.3$), in the Middle-Ages Galaxy Properties with Integral field spectroscopy (MAGPI) Survey. For each galaxy, we obtain a minimum of 11 independent visual classifications by knowledgeable classifiers. We use an extension of the standard Dawid-Skene bayesian model introducing classifier-specific confidence parameters and galaxy-specific difficulty parameters to quantify classifier confidence and infer reliable statistical confidence estimates. Selecting sub-samples of 86 bright ($r<20$ mag) high-confidence ($>0.98$) morphological classifications at redshifts ($0.2 \le z \le0.4$), we confirm the full range of morphological types is represented in MAGPI as intended in the survey design. Similarly, with a sub-sample of 82 bright high-confidence stellar kinematic classifications, we find that the rotating and non-rotating galaxies seen at low redshift are already in place at intermediate redshifts. We \textit{do not} find evidence that the kinematic morphology-density relation seen at $z\sim0$ is established at $z\sim0.3$. We suggest that galaxies without obvious stellar rotation are dynamically pre-processed sometime before $z\sim0.3$ within lower mass groups before joining denser environments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.16858-b31b1b.svg)](https://arxiv.org/abs/2502.16858) | **No Galaxy-Scale [CII] Fast Outflow in the z=6.72 Red Quasar HSC J1205$-$0000**  |
|| M. Sawamura, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-02-25*|
|*Comments*| *15 pages, 6 figures, accepted for publication in ApJ*|
|**Abstract**|            HSC 120505.09-000027.9 (J1205$-$0000) is one of the highest redshift ($z=6.72$) dust-reddened quasars (red quasars) known to date. We present an improved analysis of Atacama Large Millimeter/submillimeter Array data of the [CII] $158\ \rm{\mu m}$ line and the underlying rest-frame far-infrared (FIR) continuum emission, previously reported in Izumi et al. (2021a), toward J1205$-$0000. Red quasars are thought to be a transitional phase from an obscured starburst to a luminous blue quasar, in some cases associated with massive outflows driven by the active galactic nucleus (AGN). J1205$-$0000 has a high FIR luminosity, $L_{\mathrm{FIR}}=2.5\times 10^{12}\ L_{\odot}$ and a total IR luminosity of $L_{\mathrm{TIR}}=3.5\times 10^{12}\ L_{\odot}$, corresponding to a star formation rate (SFR) of $\sim 528\ M_{\odot}\ \mathrm{yr}^{-1}$. With the [CII]-based dynamical mass of $\sim 1 \times 10^{11}~M_\odot$, we conclude that J1205$-$0000 is hosted by a starburst galaxy. In contradiction to Izumi et al. (2021a), our improved analysis shows no hint of a broad component in the [CII] line spectrum. Thus there is no evidence for a host galaxy-scale fast [CII] outflow, despite the fact that J1205$-$0000 has fast nuclear ionized outflows seen in the rest-frame UV. We explore several scenarios for this discrepancy (e.g., early phase of AGN feedback, reliability of the [CII] line as a tracer of outflows), and we claim that it is still too early to conclude that there is no significant negative AGN feedback on star formation in this red quasar.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2502.16018/./Figures/fig3.png', 'tmp_2502.16018/./Figures/fig1.png', 'tmp_2502.16018/./Figures/fig2.png']
copying  tmp_2502.16018/./Figures/fig3.png to _build/html/
copying  tmp_2502.16018/./Figures/fig1.png to _build/html/
copying  tmp_2502.16018/./Figures/fig2.png to _build/html/
exported in  _build/html/2502.16018.md
    + _build/html/tmp_2502.16018/./Figures/fig3.png
    + _build/html/tmp_2502.16018/./Figures/fig1.png
    + _build/html/tmp_2502.16018/./Figures/fig2.png
found figures ['tmp_2502.17319/./vel_struct_dist_trail_armSgr_stream_main.png', 'tmp_2502.17319/./target_selection.png', 'tmp_2502.17319/./resolving_power.png']
copying  tmp_2502.17319/./vel_struct_dist_trail_armSgr_stream_main.png to _build/html/
copying  tmp_2502.17319/./target_selection.png to _build/html/
copying  tmp_2502.17319/./resolving_power.png to _build/html/
exported in  _build/html/2502.17319.md
    + _build/html/tmp_2502.17319/./vel_struct_dist_trail_armSgr_stream_main.png
    + _bui

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\co}{\ifmmode{\mathrm{CO}} \else CO\fi}$
$\newcommand{\coeighteen}{\ifmmode{\mathrm{C}^{18}\mathrm{O}} \else C^{18}O\fi}$
$\newcommand{\thirteenco}{\ifmmode{^{13}\mathrm{CO}} \else ^{13}CO\fi}$
$\newcommand{\hdens}{\ifmmode{n_{\mathrm{H}_{2}}} \else n_{\mathrm{H}_{2}}\fi}$
$\newcommand{\hdensLog}{\ifmmode{n_{0,\mathrm{H}_{2}}} \else n_{0,\mathrm{H}_{2}}\fi}$
$\newcommand{\tkin}{\ifmmode{\mathrm{T}_{\mathrm{kin}}} \else \mathrm{T}_{\mathrm{kin}}\fi}$
$\newcommand{\coldens}{\ifmmode{N_{\mathrm{CO}}} \else N_{\mathrm{CO}}\fi}$
$\newcommand{\width}{\ifmmode{\sigma} \else \sigma \fi}$
$\newcommand{\xthirteen}{\ifmmode{{X}_{12/13}} \else {X}_{12/13}\fi}$
$\newcommand{\xeighteen}{\ifmmode{{X}_{13/18}} \else {X}_{13/18}\fi}$
$\newcommand{\hydrogen}{\ifmmode{\mathrm{H}_{2}} \else H_{2}\fi}$
$\newcommand{\bff}{\ifmmode{\Phi} \else \Phi\fi}$
$\newcommand{\conv}{\ifmmode{\textrm{CO-to-H}_{2}}   \else CO-to-H_2 \fi}$
$\newcommand{\alphaco}{\ifmmode{\alpha_{\mathrm{CO}}}   \else \alpha_{\mathrm{CO}} \fi}$
$\newcommand{\Princeton}{\affiliation{Department of Astrophysical Sciences, Princeton University, 4 Ivy Lane, Princeton, NJ 08544, USA}}$
$\newcommand{\McMaster}{\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{\CITA}{\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}}$
$\newcommand{\OSU}{\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA}}$
$\newcommand{ÇAPP}{\affiliation{Center for Cosmology and Astroparticle Physics (CCAPP), 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\Alberta}{\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\Cologne}{\affiliation{I. Physikalisches Institut der Universität zu Köln, Zülpicher Straße 77, 50937, Köln, Germany}}$
$\newcommand{\ANU}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\Arizona}{\affiliation{Steward Observatory, University of Arizona, 933 North Cherry Avenue, Tucson, AZ 85721, USA}}$
$\newcommand{\ASIAA}{\affiliation{Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 106216, Taiwan}}$
$\newcommand{\ASTROThreeD}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\Bonn}{\affiliation{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}}$
$\newcommand{\Carnegie}{\affiliation{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{ÇA}{\affiliation{Center for Computational Astrophysics, Flatiron Institute, 162 Fifth Avenue, New York, NY 10010, USA}}$
$\newcommand{\CfA}{\affiliation{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{\CITEVA}{\affiliation{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\CNRS}{\affiliation{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\Connecticut}{\affiliation{Department of Physics, University of Connecticut, 196A Auditorium Road, Storrs, CT 06269, USA}}$
$\newcommand{\COOL}{\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{\EPFL}{\affiliation{Institute of Physics, Laboratory for galaxy evolution and spectral modelling, EPFL, Observatoire de Sauverny, Chemin Pegais 51, 1290 Versoix, Switzerland}}$
$\newcommand{\ESO}{\affiliation{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}}$
$\newcommand{\Gent}{\affiliation{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\Hawaii}{\affiliation{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}}$
$\newcommand{\Heidelberg}{\affiliation{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\IAC}{\affiliation{Instituto de Astrofísica de Canarias, C/ Vía Láctea s/n, E-38205, La Laguna, Spain}}$
$\newcommand{\ICRAR}{\affiliation{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{\INAF}{\affiliation{INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}}$
$\newcommand{\IPAC}{\affiliation{Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{\IPARC}{\affiliation{Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}}$
$\newcommand{\IRAM}{\affiliation{Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}}$
$\newcommand{\ITA}{\affiliation{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{\IWR}{\affiliation{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\JHU}{\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}}$
$\newcommand{\Kansas}{\affiliation{Department of Physics and Astronomy, University of Kansas, 1251 Wescoe Hall Drive, Lawrence, KS 66045, USA}}$
$\newcommand{\LAM}{\affiliation{Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}}$
$\newcommand{\Leiden}{\affiliation{Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}}$
$\newcommand{\Liverpool}{\affiliation{Astrophysics Research Institute, Liverpool John Moores University, IC2, Liverpool Science Park, 146 Brownlow Hill, Liverpool L3 5RF, UK}}$
$\newcommand{\Lyon}{\affiliation{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval, France}}$
$\newcommand{\Maryland}{\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, 4296 Stadium Drive, College Park, MD 20742, USA}}$
$\newcommand{\UMarry}{\affiliation{Department of Astronomy, University of Maryland, 4296 Stadium Drive, College Park, MD 20742, USA}}$
$\newcommand{\Yebes}{\affiliation{Centro de Desarrollos Tecnológicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain}}$
$\newcommand{\MPE}{\affiliation{Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}}$
$\newcommand{\MPIA}{\affiliation{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}}$
$\newcommand{\Nagoya}{\affiliation{Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}}$
$\newcommand{\NAOJ}{\affil{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}}$
$\newcommand{\Nichidai}{\affil{Department of Physics, General Studies, College of Engineering, Nihon University, 1 Nakagawara, Tokusada, Tamuramachi, Koriyama, Fukushima, 963-8642, Japan}}$
$\newcommand{\NRAO}{\affiliation{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}}$
$\newcommand{\OAN}{\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}}$
$\newcommand{\ObsParis}{\affiliation{Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}}$
$\newcommand{\Oxford}{\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}}$
$\newcommand{\Rutgers}{\affiliation{Department of Physics and Astronomy, Rutgers, the State University of New Jersey, 136 Frelinghuysen Road, Piscataway, NJ 08854, USA}}$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\STScIESA}{\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\Surrey}{\affiliation{Department of Physics, University of Surrey, Guildford GU2 7XH, UK}}$
$\newcommand{\Sydney}{\affiliation{Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}}$
$\newcommand{\TAPIR}{\affil{TAPIR, California Institute of Technology, Pasadena, CA 91125, USA}}$
$\newcommand{\Tamkang}{\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}}$
$\newcommand{\Toulouse}{\affiliation{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\Toledo}{\affiliation{University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH 43606, USA}}$
$\newcommand{\UChile}{\affiliation{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{\UCM}{\affiliation{Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}}$
$\newcommand{\UCSD}{\affiliation{Center for Astrophysics and Space Sciences, Department of Physics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{\ULL}{\affiliation{Departamento de Astrofísica, Universidad de La Laguna, Av. del Astrofísico Francisco Sánchez s/n, E-38206, La Laguna, Spain}}$
$\newcommand{\UMass}{\affiliation{University of Massachusetts—Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}}$
$\newcommand{\UVa}{\affiliation{University of Virginia, 530 McCormick Road, Charlottesville, VA 22904, USA}}$
$\newcommand{\Wyoming}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\Zurich}{\affiliation{Institute for Computational Science, University of Zürich, Winterthurerstrasse 190, 8057 Zürich, Switzerland}}$
$\newcommand{\Lund}{\affiliation{Lund Observatory, Division of Astrophysics, Department of Physics, Lund University, Box 43, SE-221 00 Lund, Sweden}}$
$\newcommand{\Rad}{\affiliation{Radcliffe Institute for Advanced Studies at Harvard University, 10 Garden Street, Cambridge, MA 02138, U.S.A.}}$
$\newcommand{\Phillips}{\affiliation{Phillips Academy, Andover, MA 01810, USA}}$</div>



<div id="title">

# Application of resolved low-$J$ multi-CO line modeling with \texttt{RADEX} to constrain the molecular gas properties in the starburst M82

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.16018-b31b1b.svg)](https://arxiv.org/abs/2502.16018)<mark>Appeared on: 2025-02-25</mark> -  _14 pages, 6 figures, accepted for publication in ApJ_

</div>
<div id="authors">

V. Zhang, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The distribution and physical conditions of molecular gas are closely linked to star formation and the subsequent evolution of galaxies. Emission from carbon monoxide (CO) and its isotopologues traces the bulk of molecular gas and provides constraints on the physical conditions through their line ratios. However, comprehensive understanding on how the particular choice of line modeling approach impacts derived molecular properties remain incomplete. Here, we study the nearby starburst galaxy M82, known for its intense star formation and molecular emission, using the large set of available multi-CO line observations. We present high-resolution ( ${\sim}85$ pc) emission of seven CO isotopologue lines, including $^{12}$ CO, $^{13}$ CO, and C $^{18}$ O from the $J = 1-0$ , $2-1$ and $3-2$ transitions. Using \texttt{RADEX} for radiative transfer modeling, we analyze M82’s molecular properties with (i) a one-zone model and (ii) a variable density model, comparing observed and simulated emissions via a minimum $\chi^2$ analysis. We find that inferred gas conditions—kinetic temperature and density—are consistent across models, with minimal statistical differences. However, due to their low critical densities ( ${<}10^{4}$ cm $^{-3}$ ), low- $J$ CO isotopologue lines do not effectively probe higher density gas prevalent in starburst environments like that of M82. Our results further imply that this limitation extends to high-redshift ( $z{\gtrapprox}1$ ) galaxies with similar conditions, where low- $J$ CO lines are inadequate for density constraints. Future studies of extreme star-forming regions like M82 will require higher- $J$ CO lines or alternative molecular tracers with higher critical densities.

</div>

<div id="div_fig1">

<img src="tmp_2502.16018/./Figures/fig3.png" alt="Fig3" width="100%"/>

**Figure 3. -** ** Histogram of 1DMax fitted parameters for $\xthirteen$, $\xeighteen$, $\coldens$, and $\tkin$ from both models.** We plot histograms of 1DMax fitted parameters, with the one-zone model (red) and the log-normal density model  (blue). The vertical dotted line is the median of the 1DMax values. In the horizontal markers above, we plot the weighted 16$^{\rm th}$-to-84$^{\rm th}$ percentile range from our results, from results found in \cite{Teng2023} for nearby barred galaxies, and from \citet{denBrok2024_M51} for M51. (*fig:overlayhist*)

</div>
<div id="div_fig2">

<img src="tmp_2502.16018/./Figures/fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** ** Optical image, CO Isotopologue Emission and Moment 0 images of M82.** On the top left, we present an optical image of M82. On the top right, we plot emission spectra for two pixels in the center regions of the galaxy. Note that we scale emission from $\thirteenco$ and $\coeighteen$ for visibility purposes. On the lower panel, we present the (1--0) and (2--1) energy transitions from each of the isotopologues $\co$, $\thirteenco$, and $\co$eighteen. \ Credit for M82 optical Image: NASA, ESA and the Hubble Heritage Team (STScI/AURA). Acknowledgment: J. Gallagher (University of Wisconsin), M. Mountain (STScI) and P. Puxley (NSF). (*fig:spectramom0*)

</div>
<div id="div_fig3">

<img src="tmp_2502.16018/./Figures/fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** ** Marginalized 1D and 2D likelihood distributions from a bright central pixel.** The red (left) corner plot is derived from our one-zone model, and the blue (right) corner plot is derived from our log-normal model. The map in the middle depicts the integrated intensity with one sightline circled from which the two corner plots are generated. (*fig:bothcorner*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.16018"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A Pristine-UNIONS view on the Galaxy: Kinematics of the         distant spur feature of the Sagittarius stream traced by Blue         Horizontal Branch stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.17319-b31b1b.svg)](https://arxiv.org/abs/2502.17319)<mark>Appeared on: 2025-02-25</mark> -  _21 pages, 12 figures, submitted to A&A, abridged abstract_

</div>
<div id="authors">

M. Bayer, et al. -- incl., <mark>N. Martin</mark>

</div>
<div id="abstract">

**Abstract:** Providing a detailed picture of the Sagittarius (Sgr) stream offers   important constraints on the build-up of the Galactic halo as well   as its gravitational potential at large radii. While several   attempts have been made to model the structure of the Sgr stream, no   model has yet been able to match all the features observed for the   stream. Moreover, for several of these features, observational   characterisation of their properties is rather limited, particularly   at large distances. The aim of this work is to investigate the kinematics of the Sgr   stream outermost spur feature using blue horizontal branch (BHB)   stars. Candidate BHB stars were selected by combining two approaches; one   capitalising on Pan-STARRS1 3 $\Pi$ _griz_ and _u_ photometry taken as part of UNIONS, the other using Pristine Survey _CaHK_ and SDSS _ugr_ photometry. Follow-up optical   spectra are obtained using ESO/VLT/FORS2 to confirm their BHB nature   and obtain line-of-sight (LOS) velocities. Of our 25 candidates, 20 stars can be confirmed as bona fide BHB   stars. Their LOS velocities, together with the 3D positions of these   stars qualitatively match well with Sgr model predictions and trace   the outer apocentre of the trailing arm and its spur feature very   nicely. The quantitative offsets that are found between our data and   the different models can be used to provide information about the   Galactic gravitational potential at large distances. We present a   first, tentative, analysis in this direction, showing that the model   of vasiliev_et_al2021 would provide better agreement with our   observations if the   enclosed mass of the Milky Way within 100 kpc   were lowered to $(5.3\!\pm\!0.4)\!\times\!10^{11} \mathrm{M}_\odot   \left(\text{versus} (5.6\!\pm\!0.4)\!\times\!10^{11}    \mathrm{M}_\odot\right)$ . Our selection of BHB stars provides a new view on the outermost   structure in 3D positions and LOS velocities of the Sgr debris.

</div>

<div id="div_fig1">

<img src="tmp_2502.17319/./vel_struct_dist_trail_armSgr_stream_main.png" alt="Fig11" width="100%"/>

**Figure 11. -** Comparison of the observational data on the LOS velocities
             in the Galactic Standard of Rest of the BHB stars (diamonds
             and stars) and predictions from the
             dierickx_loeb2017(lightgreyish points) and
             vasiliev_et_al2021 without (darkgreyish points)
             and with LMC (orange points) models in the same angular
             range across the stream. Left panel: big diamond markers
             indicate LOS velocities in Galactic Standard of Rest of BHB
             stars estimated with \lowercase{RVSpecFit} from
             the FORS2 spectra. Right panel: stars represent the same
             velocities with an applied shift as reported in Figs.
             \ref{fig:comparison_los_vel_estimates_vel_sts_main}
             and
             \ref{
               fig:comparison_los_vel_estimates_vel_sts_appendix
             }.
             The plot shows also the position of globular cluster
             NGC 2419 (filled circle) in this projection of phase
             space. The LOS velocity of NGC 2419 comes from
             vasiliev_baumgardt2021 assuming also the right
             ascension and declination provided in the
             vasiliev_baumgardt2021 catalogue while we use the
             heliocentric distance of NGC 2419 from
             baumgardt_vasiliev2021. (*fig:dist_sgr_stream_vel_struct_main*)

</div>
<div id="div_fig2">

<img src="tmp_2502.17319/./target_selection.png" alt="Fig10" width="100%"/>

**Figure 10. -** An overview of the target stars in multiple colour-colour
  	         spaces is shown. Data were gathered from multiple sources.
             These are: the Panoramic Survey Telescope and Rapid
             Response System 1 3$\Pi$ survey Data Release 2
              (ps1,  magnier_et_al2020) ,
             the Canada-France Imaging Survey, the Sloan Digital Sky
             Survey Data Release 18  (sdss_dr18) , and
             the Pristine Survey  (starkenburg_et_al2017) . The
             candidate BHB stars in our sample are colour-coded
             according to the number of their id given in
             \autoref{tab:cand_bhb_stars}. The background stars come
             from two samples. The pink markers are blue straggler
             stars from the xue_et_al2008 set while the brown
             symbols represent BHB stars from the xue_et_al2011
             dataset. (*fig:target_selection*)

</div>
<div id="div_fig3">

<img src="tmp_2502.17319/./resolving_power.png" alt="Fig12" width="100%"/>

**Figure 12. -** VLT/FORS2/600B+22 grism resolving power trends across
               detector for gathered data of arc-lamps at
               different observation times for the two used slit
               configurations. (*fig:vlt_fors2_600b22_resolving_power_trend*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.17319"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

342  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
